Will solve:

https://github.com/CONABIO/kube_sipecam_playground/issues/5

# Set up minikube and use docker image for MAD-Mex in AWS

Will follow: 

* For minikube: [minikube_sipecam/setup](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/setup#aws)

* docker image for MAD-Mex: [kube_sipecam/dockerfiles/MAD_Mex/odc_kale](https://github.com/CONABIO/kube_sipecam/tree/master/dockerfiles/MAD_Mex/odc_kale) and [minikube_sipecam/deployments/MAD_Mex](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/deployments/MAD_Mex/)


Will use [minikube_sipecam/deployments/MAD_Mex/hostpath_pv](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/deployments/MAD_Mex/hostpath_pv)

## Instance

In AWS we can select ami: `k8s-1.16-debian-buster-amd64-hvm-ebs-2020-04-27 - ami-0ab39819e336a3f3f` and instance `m5.2xlarge` with `50` gb of disk.

Use next bash script for user data, this will start minikube using `none` driver:

```
#!/bin/bash
##variables:
region=us-west-2
user=admin
name_instance=minikube
shared_volume=/shared_volume
##System update
export DEBIAN_FRONTEND=noninteractive
apt-get update -yq
##Install awscli
apt-get install -y python3-pip && pip3 install --upgrade pip
pip3 install awscli --upgrade
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region
#check if locales are ok with next lines:
echo "export LC_ALL=C.UTF-8" >> /root/.profile
echo "export LANG=C.UTF-8" >> /root/.profile
echo "export mount_point=$shared_volume" >> /root/.profile
systemctl start docker
usermod -aG docker $user
newgrp docker
#Create shared volume
mkdir $shared_volume
#kubectl installation
curl -LO "https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl"
chmod +x ./kubectl
mv ./kubectl /usr/local/bin/kubectl
kubectl version --client
#bash completion, needs to exit and enter again to take effect
#echo "source <(kubectl completion bash)" >> /root/.bashrc
#apt-get install -y bash-completion
#minikube installation
curl -Lo minikube https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 \
  && chmod +x minikube
cp minikube /usr/local/bin/
install minikube /usr/local/bin/
apt-get install conntrack -y
#kubeflow installation
cd /root && wget https://github.com/kubeflow/kfctl/releases/download/v1.0.2/kfctl_v1.0.2-0-ga476281_linux.tar.gz
tar -xvf kfctl_v1.0.2-0-ga476281_linux.tar.gz
echo "export PATH=$PATH:$(pwd)" >> /root/.profile
# Set KF_NAME to the name of your Kubeflow deployment. This also becomes the
# name of the directory containing your configuration.
# For example, your deployment name can be 'my-kubeflow' or 'kf-test'.
echo "export KF_NAME=kf-test" >> ~/.profile
echo "export BASE_DIR=/opt" >> ~/.profile
source ~/.profile
echo "export KF_DIR=${BASE_DIR}/${KF_NAME}" >> ~/.profile
```
Check installation in AWS instance with: `tail -n 15  /var/log/cloud-init-output.log`.

Change to root with `sudo su`

Ssh to instance, then:

```
CONFIG_URI="https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_k8s_istio.v1.0.2.yaml"
source ~/.profile
chmod gou+wrx -R /opt/
mkdir -p ${KF_DIR}
#minikube start
cd /root && minikube start --driver=none
#kubeflow start
cd ${KF_DIR} && kfctl apply -V -f ${CONFIG_URI}
```

Check pods and status with:

`minikube status`

```
minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured
```

`kubectl get pods -n kubeflow`

```
#all running except:
spark-operatorcrd-cleanup-2p7x2                                0/2     Completed   0          7m6s
```

To access kubeflow UI set:

```
export INGRESS_HOST=$(minikube ip)
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
echo $INGRESS_PORT
```


And go to:

```
http://<ipv4 of ec2 instance>:$INGRESS_PORT
```

# Set up postgresql instance in AWS

Will follow:

https://github.com/CONABIO/antares3-docker/tree/master/postgresql/local_deployment

```
sudo su
cd /shared_volume
dir=/shared_volume/postgresql_volume_docker
mkdir $dir

git clone https://github.com/CONABIO/antares3-docker.git $dir/antares3-docker

mkdir -p $dir/etc/postgresql
mkdir -p $dir/var/log/postgresql
mkdir -p $dir/var/lib/postgresql

docker run -v $dir/etc/postgresql:/etc/postgresql \
-v $dir/var/log/postgresql:/var/log/postgresql \
-v $dir/var/lib/postgresql:/var/lib/postgresql \
-v $dir/antares3-docker/postgresql/local_deployment/conf/:/home/postgres/conf/ \
-w /home/postgres \
-p 2225:22 -p 5432:5432 --name postgresql-local --hostname postgresql-madmex \
-dit madmex/postgresql-madmex-local:v8 /bin/bash
```

## Init files

**Next commands in jupyterlab**

`~/.datacube.conf`

```
[user]
default_environment: datacube
#default_environment: s3aio_env

[datacube]
db_hostname: 172.17.0.61 #check this with docker inspect postgresql-local|grep IPA
db_database: antares_datacube
db_username: postgres
db_password: postgres


execution_engine.use_s3: False

[s3aio_env]
db_hostname: 172.17.0.61 #check this with docker inspect postgresql-local|grep IPA
db_database: antares_datacube
db_username: postgres
db_password: postgres

#index_driver: s3aio_index

execution_engine.use_s3: False
```

`~/.antares`

```
# Django settings
SECRET_KEY=<key>
DEBUG=True
DJANGO_LOG_LEVEL=DEBUG
ALLOWED_HOSTS=
# Database
DATABASE_NAME=antares_datacube
DATABASE_USER=postgres
DATABASE_PASSWORD=postgres
DATABASE_HOST=172.17.0.61 #check this with docker inspect postgresql-local|grep IPA
DATABASE_PORT=5432
# Datacube
SERIALIZED_OBJECTS_DIR=/root/datacube_ingest/serialized_objects/
INGESTION_PATH=/root/datacube_ingest
#DRIVER=s3aio
DRIVER='NetCDF CF'
#INGESTION_BUCKET=datacube-s2-jalisco-test
# Query and download
USGS_USER=<username>
USGS_PASSWORD=<password>
SCIHUB_USER=
SCIHUB_PASSWORD=
# Misc
BIS_LICENSE=<license>
TEMP_DIR=/shared_volume/temp
#SEGMENTATION_BUCKET=<name of bucket>

```

Install antares

`pip3 install --user git+https://github.com/CONABIO/antares3.git@develop`

Init antares and datacube:

```
~/.local/bin/antares init
datacube -v system init
```


**Create spatial indexes:**

```
apt-get install postgresql-client
psql -h 172.17.0.61 -d antares_datacube -U postgres
CREATE INDEX madmex_predictobject_gix ON public.madmex_predictobject USING GIST (the_geom);
CREATE INDEX madmex_trainobject_gix ON public.madmex_trainobject USING GIST (the_geom);
```

**Check:**

`datacube -v system check`

**There are some notes that could be followed [Notes](https://github.com/CONABIO/antares3-docker/tree/master/postgresql/local_deployment#note) for docker container of postgresql**